In [19]:
# Training loop:
# 1. Create torch dataset from the data containing the query, positive, negative example.
# 2. Create a dataloader from the dataset.
# 3. Run a single batch through the model.

In [20]:
!poetry add pandas

The following packages are already present in the pyproject.toml and will be skipped:

  • pandas

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [21]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd

In [342]:
## Load data
train = pd.read_parquet("train.parquet")
test = pd.read_parquet("test.parquet")
validate = pd.read_parquet("validate.parquet")

In [23]:
print(train.shape, test.shape, validate.shape)

(82326, 6) (9650, 6) (10047, 6)


In [24]:
full_dataset = pd.concat([train, test, validate])

In [241]:
full_dataset[full_dataset["query_id"].isin([19699, 19701])]

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]


In [ ]:
# Save the Query IDs for each dataset to use on later.

In [25]:
total_size = train.shape[0] + test.shape[0] + validate.shape[0]
print(total_size)
# Rerun this code only if you want to generate new indices.
import numpy as np
import pickle

# np.random.seed(seed=999999)
# indices = np.random.randint(0, total_size, total_size)
# train_indices = indices[0:train.shape[0]]
# test_indices = indices[train.shape[0]:train.shape[0]+test.shape[0]]
# validate_indices = indices[train.shape[0]+test.shape[0]:]
# all_indices = {'train': train_indices.tolist(), 'test': test_indices.tolist(), 'validate': validate_indices.tolist()}
# print(f"Length of all indices: {sum(len(v) for k, v in all_indices.items())}")
# # write these to a pickle file

# with open('indices.pkl', 'wb') as f:
#     pickle.dump(all_indices, f)

# create train, test, validation datasets based on query_ids
# query_ids = full_dataset['query_id']
# train_query_ids = query_ids[0:train.shape[0]]
# test_query_ids = query_ids[train.shape[0]:train.shape[0]+test.shape[0]]
# validate_query_ids = query_ids[train.shape[0]+test.shape[0]:]
# print(train_query_ids.shape, test_query_ids.shape, validate_query_ids.shape)
# all_query_ids  = {'train': train_query_ids.tolist(), 'test': test_query_ids.tolist(), 'validate': validate_query_ids.tolist()}
# # write these to a pickle file

# with open('query_ids.pkl', 'wb') as f:
#     pickle.dump(all_query_ids, f)

102023


In [351]:
# Unpickle and test
with open("indices.pkl", "rb") as f:
    all_indices = pickle.load(f)

    print(f"{len(all_indices['train'])} training indices {all_indices['train'][0:10]}")
    print(f"{len(all_indices['test'])} test indices {all_indices['test'][0:10]}")
    print(
        f"{len(all_indices['validate'])} validate indices {all_indices['validate'][0:10]}"
    )

with open("query_ids.pkl", "rb") as f:
    all_query_ids = pickle.load(f)
    print(
        f"{len(all_query_ids['train'])} training query ids {all_query_ids['train'][0:10]}"
    )
    print(f"{len(all_query_ids['test'])} test query ids {all_query_ids['test'][0:10]}")
    print(
        f"{len(all_query_ids['validate'])} validate query ids {all_query_ids['validate'][0:10]}"
    )

82326 training indices [36233, 13668, 19591, 66699, 34957, 34882, 23273, 27926, 90079, 68436]
9650 test indices [27928, 5317, 97860, 83225, 90861, 42316, 42625, 98527, 74133, 95026]
10047 validate indices [94011, 72401, 60227, 70263, 35765, 47026, 31472, 3385, 21289, 545]
82326 training query ids [19699, 19700, 19701, 19702, 19703, 19704, 19705, 19706, 19707, 19708]
9650 test query ids [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
10047 validate query ids [9652, 9653, 9654, 9655, 9656, 9657, 9658, 9659, 9660, 9661]


In [361]:
# train[['passages', 'query']].iloc[0]
train_dataset = full_dataset[full_dataset.query_id.isin(all_query_ids["train"])]
test_dataset = full_dataset[full_dataset.query_id.isin(all_query_ids["test"])]
validate_dataset = full_dataset[full_dataset.query_id.isin(all_query_ids["validate"])]

train_dataset.head()["query"].iloc[0]

'what is rba'

In [362]:
print(f"{np.all(train.query_id.value_counts() == 1)}")
print(f"{np.all(train_dataset.query_id.value_counts() == 1)}")
assert np.all(train.query_id.value_counts() == 1)
assert np.all(train_dataset.query_id.value_counts() == 1)
assert np.all(validate.query_id.value_counts() == 1)

True
True


In [238]:
full_dataset.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[]
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[]


In [ ]:
from utils.data_utils import create_lookups, add_hashed_urls

In [364]:
ids_to_urls, urls_to_ids, url_to_doctext_mapping = create_lookups(full_dataset)
add_hashed_urls(train_dataset, urls_to_ids)
add_hashed_urls(test_dataset, urls_to_ids)
add_hashed_urls(validate_dataset, urls_to_ids)

dataset shape: (102023, 6)
Total number of urls: 837729
Total number of unique urls: 456487
Total number of hashed urls: 456487
Total number of queries: 102023


100%|██████████| 82326/82326 [00:00<00:00, 197706.24it/s]
/var/folders/fk/dlmj8gqd4ms2jgbkv_33skth0000gn/T/ipykernel_77110/1830918971.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, "hashed_urls"] = dataset["passages"].progress_apply(
100%|██████████| 9650/9650 [00:00<00:00, 183872.95it/s]
/var/folders/fk/dlmj8gqd4ms2jgbkv_33skth0000gn/T/ipykernel_77110/1830918971.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, "hashed_urls"] = dataset["passages"].progress_apply(
100%

In [279]:
train["passages"].iloc[0]["url"]

array(['https://en.wikipedia.org/wiki/Reserve_Bank_of_Australia',
       'https://en.wikipedia.org/wiki/Reserve_Bank_of_Australia',
       'http://acronyms.thefreedictionary.com/RBA',
       'https://www.slimvapepen.com/rebuildable-atomizer-rba/',
       'http://rba-africa.com/about/what-is-rba/',
       'http://resultsleadership.org/what-is-results-based-accountability-rba/',
       'http://rba-africa.com/about/what-is-rba/',
       'http://searchsecurity.techtarget.com/definition/risk-based-authentication-RBA',
       'https://www.slimvapepen.com/rebuildable-atomizer-rba/',
       'http://www.rbaconsulting.com/'], dtype=object)

In [365]:
print(url_to_doctext_mapping["51456f6eae52e682e8cc2877123547df"])
print(urls_to_ids["http://rba-africa.com/about/what-is-rba/"])
print(ids_to_urls["c2f05ed2ec73e1b97799c70e4cdbed8f"])

For example, you could contribute $2,500 to a Roth IRA with one provider and $3,000 with a second provider. You cannot contribute $5,500 to the first Roth IRA and $5,500 to the second provider. Your total contribution limit is $5,500 for all accounts for the tax year. Individuals may contribute money to a Roth IRA from January 1 of the current tax year to April 15 of the following tax year. Contributions are made with post-tax dollars. You pay tax today so that in retirement no tax is charged to you in retirement.
c2f05ed2ec73e1b97799c70e4cdbed8f
http://rba-africa.com/about/what-is-rba/


In [ ]:
from tqdm import tqdm

tqdm.pandas()

In [366]:
master_url_id_set = np.array(
    list(set(ids_to_urls.keys()))
)  # Define or load your master_url_id_set here

# Triples

In [131]:
# Splitting the dataset for demonstration; in practice, adjust based on your dataset size
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial
import utils
import importlib

importlib.reload(utils)
importlib.reload(utils.data_utils)
from utils.data_utils import add_negative_samples

In [367]:
training_dataset_copy = train_dataset.copy()
add_negative_samples(training_dataset_copy, ids_to_urls)

/Users/sid/workspace/mlx-week3/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
Processing Batches: 100%|██████████| 10/10 [11:02<00:00, 66.24s/it]  


In [368]:
assert np.all(training_dataset_copy.query_id.value_counts() == 1)

In [370]:
test_dataset_copy = test_dataset.copy()
add_negative_samples(test_dataset_copy, ids_to_urls)

/Users/sid/workspace/mlx-week3/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
Processing Batches: 100%|██████████| 10/10 [01:18<00:00,  7.88s/it]


In [371]:
# Sanity check
assert np.all(test_dataset_copy.query_id.value_counts() == 1)

In [372]:
validate_dataset_copy = validate_dataset.copy()
add_negative_samples(validate_dataset_copy, ids_to_urls)

/Users/sid/workspace/mlx-week3/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
Processing Batches: 100%|██████████| 10/10 [01:29<00:00,  8.98s/it]


In [373]:
# Sanity check
assert np.all(validate_dataset_copy.query_id.value_counts() == 1)

In [374]:
triple_columns = ["query_id", "query", "hashed_urls", "negative_sample_urls"]
training_dataset_copy[triple_columns].to_parquet(
    "training_dataset_triplets.parquet", engine="pyarrow"
)

In [375]:
test_dataset_copy[triple_columns].to_parquet(
    "test_dataset_triplets.parquet", engine="pyarrow"
)

In [376]:
validate_dataset_copy[triple_columns].to_parquet(
    "validate_dataset_triplets.parquet", engine="pyarrow"
)

In [377]:
# There is a unit test to test this in the tests/ dir.
s1 = list([len(i[1]["negative_sample_urls"]) for i in training_dataset_copy.iterrows()])
s2 = list([len(i[1]["hashed_urls"]) for i in training_dataset_copy.iterrows()])
print(len(s1))
print(len(s2))
print(s1[0:10])
print(s2[0:10])
assert s1 == s2

82326
82326
[7, 6, 6, 6, 7, 8, 4, 6, 7, 5]
[7, 6, 6, 6, 7, 8, 4, 6, 7, 5]


# Triples -> Embeddings


In [159]:
import sentencepiece as spm
from gensim.models import Word2Vec
import multiprocessing

In [160]:
sp = spm.SentencePieceProcessor()
sp.load("spm_AllTexts.model")

True

In [161]:
EMBEDDING_SIZE = 128

In [162]:
w2v_model = Word2Vec(
    min_count=20,
    window=10,
    vector_size=EMBEDDING_SIZE,
    sample=6e-5,
    alpha=0.03,
    min_alpha=0.0007,
    negative=20,
    workers=multiprocessing.cpu_count() - 1,
)

In [163]:
w2v_model = Word2Vec.load("word2vec.model")

### Convert the tripleID's to embeddings

In [283]:
import importlib

importlib.reload(utils)
importlib.reload(utils.data_utils)
from utils.data_utils import triples_to_embeddings

In [378]:
training_triples = triples_to_embeddings(
    training_dataset_copy[triple_columns],
    url_to_doctext_mapping,
    sp,
    w2v_model,
    embedding_size=EMBEDDING_SIZE,
)

 26%|██▌       | 21255/82326 [00:45<02:20, 435.01it/s]

In [285]:
testing_triples = triples_to_embeddings(
    test_dataset_copy[triple_columns],
    url_to_doctext_mapping,
    sp,
    w2v_model,
    embedding_size=EMBEDDING_SIZE,
)

100%|██████████| 9650/9650 [00:21<00:00, 450.77it/s]


In [286]:
validation_triples = triples_to_embeddings(
    validate_dataset_copy[triple_columns],
    url_to_doctext_mapping,
    sp,
    w2v_model,
    embedding_size=EMBEDDING_SIZE,
)

100%|██████████| 10047/10047 [00:23<00:00, 424.82it/s]


In [291]:
[tuple(i) for i in training_triples[0:1]]

[(55966,
  array([-2.1337132 ,  0.84064513,  0.850455  ,  0.07822343,  0.49461168,
         -0.21640351, -0.15194969, -0.71110517,  0.34412065, -1.5072731 ,
         -0.9706612 , -0.0161262 ,  2.809772  ,  1.0979289 , -0.8815194 ,
         -0.4938795 , -1.4507179 , -0.3244747 , -1.2726836 ,  0.3466394 ,
          1.8815925 ,  2.5044932 , -0.66266185,  0.7998357 , -0.02810423,
         -0.15230922, -3.0367901 , -1.2586932 ,  1.8494408 , -0.00574902,
         -0.73826456,  1.8830578 , -1.4142417 ,  0.17999592,  1.2118864 ,
          2.014254  , -2.0892997 , -0.47383612,  0.973999  ,  0.3915617 ,
         -0.09133792, -0.66346806,  0.349106  ,  0.15383756, -1.0399258 ,
         -2.2609715 ,  1.2259423 , -1.7501469 , -1.8145696 ,  2.6699157 ,
         -0.49966836, -0.5354726 ,  1.6364126 , -0.7197745 ,  0.20664302,
          0.7863251 , -1.4608712 ,  0.75478077, -1.1111755 , -0.66095424,
         -0.14250354,  0.8046602 , -2.7221677 ,  0.9697246 ,  1.3395002 ,
          0.43308544,  1.7812

In [293]:
# Pickle these to use for training
import pickle

with open("training_triples.pkl", "wb") as f:
    pickle.dump([tuple(i) for i in training_triples], f)
with open("testing_triples.pkl", "wb") as f:
    pickle.dump([tuple(i) for i in testing_triples], f)
with open("validation_triples.pkl", "wb") as f:
    pickle.dump([tuple(i) for i in validation_triples], f)

### Create Dataloaders

In [296]:
# Create a torch dataset from the triples
from torch.utils.data import Dataset, DataLoader

In [332]:
len(training_triples)

82326

In [338]:
training_dataset_copy.shape

(82326, 9)

In [350]:
# training_dataset_copy[(training_dataset_copy.query_id == 55966)]
np.all(train.query_id.value_counts().values == 1)
training_dataset_copy.query_id.value_counts()

query_id
62367    7
10515    7
16294    7
5458     6
38694    6
        ..
26998    1
13036    1
37958    1
52395    1
36237    1
Name: count, Length: 56417, dtype: int64

In [337]:
### How does each triple look before we feed it into the dataset
for i, training_triple in enumerate(training_triples[0:5]):
    print(f"{i}th triple with query_id: {training_triple[0]}")
    print(
        f"\t#Relevant Docs = {len(training_triple.relevant_doc_embedding)}, Irrelevant Docs = {len(training_triple.irrelevant_doc_embedding)}"
    )
    print(
        f"\tAll are of shaped: {training_triple.relevant_doc_embedding[0].shape}? {all([i.shape == training_triple.relevant_doc_embedding[0].shape for i in training_triple.relevant_doc_embedding])}"
    )
    # print((training_dataset_copy.query_id == training_triple.query_id).sum())
    print(
        f"\tThe query text is {training_dataset_copy[training_dataset_copy.query_id == training_triple.query_id]['query'].values}"
    )

0th triple with query_id: 55966
	#Relevant Docs = 7, Irrelevant Docs = 7
	All are of shaped: (128,)? True
Training ID: 55966
	The query text is ['is there a limit to roth ira contributions'
 'is there a limit to roth ira contributions'
 'is there a limit to roth ira contributions']
1th triple with query_id: 33374
	#Relevant Docs = 9, Irrelevant Docs = 9
	All are of shaped: (128,)? True
Training ID: 33374
	The query text is ['what is hydrocodone liquid used for'
 'what is hydrocodone liquid used for'
 'what is hydrocodone liquid used for'
 'what is hydrocodone liquid used for'
 'what is hydrocodone liquid used for']
2th triple with query_id: 39304
	#Relevant Docs = 5, Irrelevant Docs = 5
	All are of shaped: (128,)? True
Training ID: 39304
	The query text is ['calcium supplements causing muscle pain'
 'calcium supplements causing muscle pain']
3th triple with query_id: 86472
	#Relevant Docs = 5, Irrelevant Docs = 5
	All are of shaped: (128,)? True
Training ID: 86472
	The query text is ['

In [301]:
class TripletDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.unrolled_data = [
            (qe, rde[i], ide[i])
            for _, qe, rde, ide in self.data
            for i in range(len(rde))
        ]

    def __len__(self):
        return len(self.unrolled_data)

    def __getitem__(self, idx):
        return self.unrolled_data[idx]

In [313]:
training_triplet_dataset = TripletDataset(training_triples)
test_triplet_dataset = TripletDataset(testing_triples)
validate_triplet_dataset = TripletDataset(validation_triples)

In [303]:
training_dataloader = DataLoader(training_triplet_dataset, batch_size=32, shuffle=False)

In [312]:
for i, (q, rel, irrel) in enumerate(training_dataloader):
    print(q.shape)
    print(rel.shape)
    print(irrel.shape)

    break

torch.Size([32, 128])
torch.Size([32, 128])
torch.Size([32, 128])


In [ ]:
# Feed the data through two tower RNN, simple RNNs to
# start. No LSTM or GRUs with a margin and triplet loss.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


class TwoTowerRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TwoTowerRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn1 = nn.RNN(input_size, hidden_size)
        self.rnn2 = nn.RNN(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, q, rel, irrel):

        q_out, q_hidden = self.rnn1(q)
        rel_out, rel_hidden = self.rnn2(rel)
        irrel_out, irrel_hidden = self.rnn2(irrel)
        return q_out, rel_out, irrel_out

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)